In [1]:
import pandas as pd
from sqlalchemy import create_engine
# Import PostgreSQL username and password
from config import username, password
# Import pycountry to add country id codes
import pycountry

### Extract CSVs into DataFrames

In [2]:
# Import csv into happiness_df
happiness_file = "world-happiness-report.csv"
happiness_df = pd.read_csv(happiness_file)

### Transform DataFrames

In [3]:
# Filter happiness_df for years above 2011
happiness_df = happiness_df[happiness_df.year >= 2011]
happiness_df

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
3,Afghanistan,2011,3.832,7.620,0.521,51.92,0.496,0.162,0.731,0.611,0.267
4,Afghanistan,2012,3.783,7.705,0.521,52.24,0.531,0.236,0.776,0.710,0.268
5,Afghanistan,2013,3.572,7.725,0.484,52.56,0.578,0.061,0.823,0.621,0.273
6,Afghanistan,2014,3.131,7.718,0.526,52.88,0.509,0.104,0.871,0.532,0.375
7,Afghanistan,2015,3.983,7.702,0.529,53.20,0.389,0.080,0.881,0.554,0.339
...,...,...,...,...,...,...,...,...,...,...,...
1944,Zimbabwe,2016,3.735,7.984,0.768,54.40,0.733,-0.095,0.724,0.738,0.209
1945,Zimbabwe,2017,3.638,8.016,0.754,55.00,0.753,-0.098,0.751,0.806,0.224
1946,Zimbabwe,2018,3.616,8.049,0.775,55.60,0.763,-0.068,0.844,0.710,0.212
1947,Zimbabwe,2019,2.694,7.950,0.759,56.20,0.632,-0.064,0.831,0.716,0.235


In [4]:
# Rename and select the column headers
#happiness_cols = ["Country name", "year", "Life Ladder", "Social support", "Freedom to make life choices", "Perceptions of corruption"]
happiness_transformed = happiness_df.rename(columns={"Country name": "country",
                                                    "year": "year",
                                                    "Life Ladder": "happiness_rating",
                                                     "Log GDP per capita": "gdp_per_capita",
                                                    "Social support": "social_support",
                                                     "Healthy life expectancy at birth": "life_expectancy",
                                                    "Freedom to make life choices": "freedom",
                                                     "Generosity": "generosity",
                                                    "Perceptions of corruption": "corruption"})
#happiness_df_cols = ["country", "happiness_rating", "social_support", "freedom", "corruption"]
#happiness_transformed = happiness_transformed[happiness_df_cols].copy()
happiness_transformed = happiness_transformed.drop(["Positive affect", "Negative affect"], axis=1).reset_index()
happiness_transformed = happiness_transformed.drop(["index"], axis=1)
happiness_transformed

,country,year,happiness_rating,gdp_per_capita,social_support,life_expectancy,freedom,generosity,corruption
0,Afghanistan,2011,3.832,7.620,0.521,51.92,0.496,0.162,0.731
1,Afghanistan,2012,3.783,7.705,0.521,52.24,0.531,0.236,0.776
2,Afghanistan,2013,3.572,7.725,0.484,52.56,0.578,0.061,0.823
3,Afghanistan,2014,3.131,7.718,0.526,52.88,0.509,0.104,0.871
4,Afghanistan,2015,3.983,7.702,0.529,53.20,0.389,0.080,0.881
...,...,...,...,...,...,...,...,...,...
1378,Zimbabwe,2016,3.735,7.984,0.768,54.40,0.733,-0.095,0.724
1379,Zimbabwe,2017,3.638,8.016,0.754,55.00,0.753,-0.098,0.751
1380,Zimbabwe,2018,3.616,8.049,0.775,55.60,0.763,-0.068,0.844
1381,Zimbabwe,2019,2.694,7.950,0.759,56.20,0.632,-0.064,0.831


In [5]:
# Define a function to add the alpha_2 id code corresponding to each country name
def findCountry (country_name):
    try:
        return pycountry.countries.get(name=country_name).alpha_2
    except:
        return ("Not found")

In [19]:
happiness_transformed["id"] = happiness_transformed.apply(lambda row: findCountry(row.country), axis = 1)
happiness_master = happiness_transformed.drop(['id'], axis = 1)
happiness_master

,country,year,happiness_rating,gdp_per_capita,social_support,life_expectancy,freedom,generosity,corruption
0,Afghanistan,2011,3.832,7.620,0.521,51.92,0.496,0.162,0.731
1,Afghanistan,2012,3.783,7.705,0.521,52.24,0.531,0.236,0.776
2,Afghanistan,2013,3.572,7.725,0.484,52.56,0.578,0.061,0.823
3,Afghanistan,2014,3.131,7.718,0.526,52.88,0.509,0.104,0.871
4,Afghanistan,2015,3.983,7.702,0.529,53.20,0.389,0.080,0.881
...,...,...,...,...,...,...,...,...,...
1378,Zimbabwe,2016,3.735,7.984,0.768,54.40,0.733,-0.095,0.724
1379,Zimbabwe,2017,3.638,8.016,0.754,55.00,0.753,-0.098,0.751
1380,Zimbabwe,2018,3.616,8.049,0.775,55.60,0.763,-0.068,0.844
1381,Zimbabwe,2019,2.694,7.950,0.759,56.20,0.632,-0.064,0.831


In [7]:
# Create a master country list with alpha_2 id codes
country_id_cols = ["id", "country"]
country_id = happiness_transformed[country_id_cols].copy().drop_duplicates()
#Filter the not found countries to manually add the alpha_2 codes
country_id_missing = country_id[country_id.id == "Not found"]
country_id_missing = country_id_missing.reset_index(drop = True)

In [8]:
#Export the missing one
country_id_missing.to_csv('country_id_missing.csv')

### Create database connection

In [9]:
country_id_filled = pd.read_csv('country_id_filled.csv')
country_id_filled

,id,country
0,BO,Bolivia
1,CD,Congo (Brazzaville)
2,CD,Congo (Kinshasa)
3,CZ,Czech Republic
4,HK,Hong Kong S.A.R. of China
5,IR,Iran
6,CI,Ivory Coast
7,XK,Kosovo
8,LA,Laos
9,MD,Moldova


In [10]:
country_id_complete = pd.merge(country_id, country_id_filled, on = "country", how = "outer")
#country_id_complete = country_id_complete.drop(["id_x"], axis = 1)
#country_id_complete = country_id_complete.rename(columns = {"id_y": "id"})
country_id_complete

,id_x,country,id_y
0,AF,Afghanistan,NaN
1,AL,Albania,NaN
2,DZ,Algeria,NaN
3,AO,Angola,NaN
4,AR,Argentina,NaN
...,...,...,...
159,Not found,Venezuela,VE
160,Not found,Vietnam,VN
161,YE,Yemen,NaN
162,ZM,Zambia,NaN


In [16]:
country_id_complete["id"] = country_id_complete["id_y"].fillna(country_id_complete["id_x"])
country_id_master = country_id_complete.drop(["id_x", "id_y"], axis = 1)
country_id_master

,country,id
0,Afghanistan,AF
1,Albania,AL
2,Algeria,DZ
3,Angola,AO
4,Argentina,AR
...,...,...
159,Venezuela,VE
160,Vietnam,VN
161,Yemen,YE
162,Zambia,ZM
